In [ ]:
! pip install dspy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does n

In [ ]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
import dspy

class SQL(dspy.Signature):
    """Generate SQL code based on a given query and table schema."""
    query: str = dspy.InputField()
    table_schema: str = dspy.InputField()
    answer: str = dspy.OutputField()
class SQLGenerator(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predict = dspy.Predict(SQL)

    def forward(self, query, table_schema):
        return self.predict(query=query, table_schema=table_schema)

table_schema="employee(id,name,salary)"
trainset = [
    dspy.Example(query="what is the price of apple?",table_schema="employee(id,name,salary)", answer="SELECT name FROM employee;").with_inputs("query","table_schema"),
    dspy.Example(query="what is the target price of tesla",table_schema="employee(id,name,salary)",answer="SELECT salary FROM employee WHERE id = 101;").with_inputs("query","table_schema"),
    dspy.Example(query="",table_schema= "what is the walmart target price?.",answer="SELECT * FROM employee WHERE salary > 50000;").with_inputs("query","table_schema"),
    dspy.Example(query="what is the market cap of IT sector in 2025?.",table_schema="employee(id,name,salary)", answer="SELECT COUNT(*) FROM employee;").with_inputs("query","table_schema"),
    dspy.Example(query="what is the rating for LTI?",table_schema="employee(id,name,salary)", answer="SELECT AVG(salary) AS average_salary FROM employee;").with_inputs("query","table_schema"),
    dspy.Example(query="what is the rating for IT sector?",table_schema="employee(id,name,salary)", answer="SELECT name FROM employee WHERE salary = (SELECT DISTINCT salary FROM employee ORDER BY salary DESC LIMIT 1 OFFSET 1)").with_inputs("query","table_schema"),
]

lm = dspy.LM("openai/gpt-4o-mini")
dspy.settings.configure(lm=lm)


from dspy.evaluate import answer_exact_match as metric
from dspy.teleprompt import BootstrapFewShot

optimizer = BootstrapFewShot(
    metric=metric,
    max_bootstrapped_demos=6,
    max_labeled_demos=6,
    max_rounds=10,
)


_optimized = optimizer.compile(student=SQLGenerator(),trainset=trainset)

100%|██████████| 6/6 [00:28<00:00,  4.78s/it]

Bootstrapped 3 full traces after 5 examples for up to 10 rounds, amounting to 33 attempts.


In [ ]:
_optimized(query="What is the third highest salary of employee?",table_schema="employee(id,employee_name,salary)")

Prediction(
    answer='SELECT DISTINCT salary FROM employee ORDER BY salary DESC LIMIT 1 OFFSET 2;'
)

In [ ]:
lm.inspect_history()





[2025-07-03T17:48:36.559269]

System message:

Your input fields are:
1. `query` (str): 
2. `table_schema` (str):
Your output fields are:
1. `answer` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## query ## ]]
{query}

[[ ## table_schema ## ]]
{table_schema}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Generate SQL code based on a given query and table schema.


User message:

[[ ## query ## ]]
What are the names of all employees?

[[ ## table_schema ## ]]
employee(id,name,salary)


Assistant message:

[[ ## answer ## ]]
SELECT name FROM employee;


User message:

[[ ## query ## ]]
What is the salary of the employee with ID 101?

[[ ## table_schema ## ]]
employee(id,name,salary)


Assistant message:

[[ ## answer ## ]]
SELECT salary FROM employee WHERE id = 101;


User message:

[[ ## query ## ]]
List all employees with a salary greater than 50,000.

[[ #

In [ ]:
import dspy

class SQL(dspy.Signature):
    """Generate SQL code based on a given query and table schema."""
    query: str = dspy.InputField()
    table_schema: str = dspy.InputField()
    answer: str = dspy.OutputField()
class SQLGenerator(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predict = dspy.Predict(SQL)

    def forward(self, query, table_schema):
        return self.predict(query=query, table_schema=table_schema)

table_schema="""cio(rating_text VARCHAR(10), -- 'Buy', 'Neutral', 'Sell'
    rating_type VARCHAR(50), -- e.g., '12-Month'
    rating_as_of_date DATE,
    suspended BOOLEAN,
    restricted BOOLEAN,
    wire_issuer_name VARCHAR(100),
    wire_issuer_full_name VARCHAR(255),
    ris_code VARCHAR(50),
    bloomberg_symbol VARCHAR(50),
    is_equity_researched BOOLEAN,
    status VARCHAR(20), -- e.g., 'Covered'
    market_cap_usd DECIMAL(20, 2),
    price DECIMAL(18, 4),
    sedol VARCHAR(12),
    isin VARCHAR(20),
    cusip VARCHAR(12),
    price_target DECIMAL(18, 4),
    price_target_as_of_date DATE)"""

trainset = [
    dspy.Example(query="what is the price of apple?",table_schema=table_schema, answer=""""SELECT CASE WHEN suspension 'Y' THEN 'No Information can be provided about this security as it is suspended.' ELSE CAST (price AS VARCHAR) END as result FROM cio WHERE company_name ilike '%apple% ORDER BY rating_change date DESC LIMIT 1;""").with_inputs("query","table_schema"),
    dspy.Example(query="What is target price for tesla?",table_schema=table_schema,answer="""SELECT CASE WHEN suspension = 'Y' THEN 'No Information can be provided about this security as it is suspend .' ELSE 'The Chief Investment Office does not provide information about price targets.' END as result FROM cio WHERE company_name ILIKE '%tesla%' ORDER BY rating_change_date DESC LIMIT 1;""").with_inputs("query","table_schema"),
    dspy.Example(query="what is the walmart target price?",table_schema=table_schema,answer="""SELECT CASE WHEN suspended TRUE THEN 'No Information can be provided about this security' WHEN restricted =TRUE THEN 'This is a restricted security. We are not able to provide ratings or price target information on this security at this time.' ELSE CAST (price_target as VARCHAR) END as result FROM cio.listings_ratings_view_complete WHERE wire_issuer_name ILIKE '% almart% or wire issuer_full_name ilike '%walmart%' or ris_code ilike '%walmart% or
bloomberg_symbol ilike '%walmart%' ORDER BY price_as_of_date DESC LIMIT 1;""").with_inputs("query","table_schema"),
    dspy.Example(query="what is the rating for LTI?",table_schema=table_schema, answer="""SELECT CASE WHEN suspension = 'Y' THEN 'No Information can be provided about this security as it is suspended.' ELSE CAST (ranking_description as VARCHAR) END as result FROM cio WHERE company_name ilike '%lti%' ORDER BY rating_change_date DESC LIMIT 1;""").with_inputs("query","table_schema"),
    dspy.Example(query="What is target price for Google?",table_schema=table_schema, answer="""SELECT CASE WHEN suspended = TRUE THEN 'No Information can be provided about this security' WHEN restricted = TRUE THEN 'This is a restricted security. We are not able to provide ratings or price target information on this security at this time.' ELSE CAST (price_target as VARCHAR) END as result FROM cio.listings_ratings_view_complete WHERE wire_issuer_name ILIKE '%google% or wire_issuer_full_name ilike '%google% or ris_code ilike '%google%' or bloomberg_symbol ilike '%google%' ORDER BY
price_as_of_date DESC LIMIT 1""").with_inputs("query","table_schema"),
    dspy.Example(query="what is the rating for IT sector?",table_schema=table_schema, answer="""SELECT CASE WHEN suspension 'Y' THEN 'No Information can be provided about this security as it is suspended.' ELSE CAST (ranking_description AS VARCHAR) END AS result FROM cio WHERE company_name ILIKE '%IT%' ORDER BY rating_change_date DESC LIMIT 1;""").with_inputs("query","table_schema"),
]

In [ ]:
lm = dspy.LM("openai/gpt-4o-mini")
dspy.settings.configure(lm=lm)


from dspy.evaluate import answer_exact_match as metric
from dspy.teleprompt import BootstrapFewShot

optimizer = BootstrapFewShot(
    metric=metric,
    max_bootstrapped_demos=6,
    max_labeled_demos=6,
    max_rounds=10,
)


_optimized = optimizer.compile(student=SQLGenerator(),trainset=trainset)

100%|██████████| 6/6 [02:01<00:00, 20.21s/it]

Bootstrapped 0 full traces after 5 examples for up to 10 rounds, amounting to 60 attempts.


In [ ]:
lm.inspect_history()





[2025-07-04T09:51:59.996838]

System message:

Your input fields are:
1. `query` (str): 
2. `table_schema` (str):
Your output fields are:
1. `answer` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## query ## ]]
{query}

[[ ## table_schema ## ]]
{table_schema}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Generate SQL code based on a given query and table schema.


User message:

[[ ## query ## ]]
what is the rating for LTI?

[[ ## table_schema ## ]]
cio(rating_text VARCHAR(10), -- 'Buy', 'Neutral', 'Sell'
    rating_type VARCHAR(50), -- e.g., '12-Month'
    rating_as_of_date DATE,
    suspended BOOLEAN,
    restricted BOOLEAN,
    wire_issuer_name VARCHAR(100),
    wire_issuer_full_name VARCHAR(255),
    ris_code VARCHAR(50),
    bloomberg_symbol VARCHAR(50),
    is_equity_researched BOOLEAN,
    status VARCHAR(20), -- e.g., 'Covered'
    market_cap_usd DEC

In [ ]:
_optimized(query="what is the rating of walmart?",table_schema=table_schema)

Prediction(
    answer="SELECT CASE WHEN suspended = TRUE THEN 'No Information can be provided about this security as it is suspended.' ELSE CAST (rating_text AS VARCHAR) END AS result FROM cio WHERE wire_issuer_name ILIKE '%walmart%' OR wire_issuer_full_name ILIKE '%walmart%' ORDER BY rating_as_of_date DESC LIMIT 1;"
)

In [ ]:
lm.inspect_history(n=1)





[2025-07-04T09:55:14.036581]

System message:

Your input fields are:
1. `query` (str): 
2. `table_schema` (str):
Your output fields are:
1. `answer` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## query ## ]]
{query}

[[ ## table_schema ## ]]
{table_schema}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Generate SQL code based on a given query and table schema.


User message:

[[ ## query ## ]]
what is the price of apple?

[[ ## table_schema ## ]]
cio(rating_text VARCHAR(10), -- 'Buy', 'Neutral', 'Sell'
    rating_type VARCHAR(50), -- e.g., '12-Month'
    rating_as_of_date DATE,
    suspended BOOLEAN,
    restricted BOOLEAN,
    wire_issuer_name VARCHAR(100),
    wire_issuer_full_name VARCHAR(255),
    ris_code VARCHAR(50),
    bloomberg_symbol VARCHAR(50),
    is_equity_researched BOOLEAN,
    status VARCHAR(20), -- e.g., 'Covered'
    market_cap_usd DEC

structured and unstructred

In [ ]:
from typing import Literal
class Router(dspy.Signature):
    """Classify the given query is used to determine whether the answer should be retrieved from an unstructured or structured dataset"""
    query: str = dspy.InputField()
    answer: Literal["Unstructured", "Structured"] = dspy.OutputField()
class QueryRouter(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predict = dspy.Predict(Router)
    def forward(self, query):
        return self.predict(query=query)


trainset = [
    dspy.Example(query="what is the price of apple?",  answer="Structured").with_inputs("query"),
    dspy.Example(query="What is target price for tesla?",answer="Structured").with_inputs("query"),
    dspy.Example(query="What is CIO view on Evolution Mining?",answer="Unstructured").with_inputs("query"),
    dspy.Example(query="What is our view on Nvidia?", answer="Unstructured").with_inputs("query"),
    dspy.Example(query="What is target price for Google?", answer="Structured").with_inputs("query"),
    dspy.Example(query="What is ranking description for Meta?", answer="Unstructured").with_inputs("query"),
]

# 4. Configure DSPy LM
lm = dspy.LM("openai/gpt-4o-mini") # Or use dspy.HFModel
dspy.settings.configure(lm=lm)


from dspy.evaluate import answer_exact_match as metric
from dspy.teleprompt import BootstrapFewShot

optimizer = BootstrapFewShot(
    metric=metric,
    max_bootstrapped_demos=6,
    max_labeled_demos=6,
    max_rounds=10,
)


_optimized = optimizer.compile(QueryRouter(), trainset=trainset)

100%|██████████| 6/6 [00:05<00:00,  1.05it/s]

Bootstrapped 6 full traces after 5 examples for up to 10 rounds, amounting to 6 attempts.


In [ ]:
_optimized(query="What is the target price of samsung?")

Prediction(
    answer='Structured'
)

In [ ]:
lm.inspect_history()





[2025-07-03T18:25:27.392497]

System message:

Your input fields are:
1. `query` (str):
Your output fields are:
1. `answer` (Literal['Unstructured', 'Structured']):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## query ## ]]
{query}

[[ ## answer ## ]]
{answer}        # note: the value you produce must exactly match (no extra characters) one of: Unstructured; Structured

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Classify the given query is used to determine whether the answer should be retrieved from an unstructured or structured dataset


User message:

[[ ## query ## ]]
what is the price of apple?


Assistant message:

[[ ## answer ## ]]
Structured


User message:

[[ ## query ## ]]
What is target price for tesla?


Assistant message:

[[ ## answer ## ]]
Structured


User message:

[[ ## query ## ]]
What is CIO view on Evolution Mining?


Assistant message:

[[ ## answer ## ]]
Unstructu

In [ ]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
config = dict(max_bootstrapped_demos=6, max_labeled_demos=6, num_candidate_programs=10, num_threads=4)
teleprompter = BootstrapFewShotWithRandomSearch(metric=metric, **config)
optimized_prompt = teleprompter.compile(QueryRouter(), trainset=trainset)

Going to sample between 1 and 6 traces per predictor.
Will attempt to bootstrap 10 candidate sets.
Average Metric: 6.00 / 6 (100.0%): 100%|██████████| 6/6 [00:00<00:00, 1214.22it/s]

2025/07/03 18:27:43 INFO dspy.evaluate.evaluate: Average Metric: 6 / 6 (100.0%)



New best score: 100.0 for seed -3
Scores so far: [100.0]
Best score so far: 100.0
Average Metric: 6.00 / 6 (100.0%): 100%|██████████| 6/6 [00:00<00:00, 925.08it/s]

2025/07/03 18:27:43 INFO dspy.evaluate.evaluate: Average Metric: 6 / 6 (100.0%)



Scores so far: [100.0, 100.0]
Best score so far: 100.0


...


Scores so far: [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
Best score so far: 100.0


 67%|██████▋   | 4/6 [00:00<00:00, 381.73it/s]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Average Metric: 6.00 / 6 (100.0%): 100%|██████████| 6/6 [00:00<00:00,  7.63it/s]

2025/07/03 18:27:53 INFO dspy.evaluate.evaluate: Average Metric: 6 / 6 (100.0%)



Scores so far: [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
Best score so far: 100.0
13 candidate programs found.


In [ ]:
optimized_prompt(query="What is the impact of israel iran war on global market?")

Prediction(
    answer='Unstructured'
)

In [ ]:
lm.inspect_history()





[2025-07-03T18:29:26.676904]

System message:

Your input fields are:
1. `query` (str):
Your output fields are:
1. `answer` (Literal['Unstructured', 'Structured']):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## query ## ]]
{query}

[[ ## answer ## ]]
{answer}        # note: the value you produce must exactly match (no extra characters) one of: Unstructured; Structured

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Classify the given query is used to determine whether the answer should be retrieved from an unstructured or structured dataset


User message:

[[ ## query ## ]]
What is the impact of israel iran war on global market?

Respond with the corresponding output fields, starting with the field `[[ ## answer ## ]]` (must be formatted as a valid Python Literal['Unstructured', 'Structured']), and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## answer ## ]]
Unst